# <center> **Brief Video**

**Détection de mouvements répétitifs dans une vidéo - Déterminer le nombre de pompes**

- OpenCV pour extraire chaque frame puis les soumettre à Mediapipe,<br>
- Récupérer les landmarks (ou bien créer de nouvelles features en calculant l'angle entre 3 points),
- Features engineering pour déceler les variables plsu les plus pertinentes,
- Détecter les segments de vidéo où il y a un mouvement répétitif (étude de la stationnarité via un fenêtrage)


**landmarks** : repères corporels -> points d'intérêts constituant le squelette.

## **Sources**


**Remarque** : Run this notebook with environnement "time_series_video", in this env mediapipe was installed on python=3.12.7

**OpenCV**<br>
<code>pip install opencv-python</code><br>
<code>import cv2</code><br>
https://www.geeksforgeeks.org/opencv-python-tutorial/<br>
https://www.tresfacile.net/la-bibliotheque-opencv-python/<br>
<br>
<br>


**mediapipe**<br>
https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker?hl=fr<br>
<br>
<br>
<br>
<br>




## **Imports**

In [ ]:
%reset

In [1]:
import os
import cv2
import mediapipe as mp
import csv
import script

# Path of the video
video_path = "videos/seq_a.mov"

In [3]:
# CODE BERTRAND

# Create an object "VideoCapture" with the video 
cap = cv2.VideoCapture(video_path)

# cap.isOpened(): Returns true if video capturing has been initialized already
while cap.isOpened():

    # cap.read(): Grabs, decodes and returns the next video frame.
    ret, frame = cap.read()

    # If there is no more frame BREAK THE LOOP
    if not ret:
        break

    # Display the frame
    cv2.imshow("Video", frame)

    # waitkey(delay): wait for 'delay' milisec AND
    # if a key is pressed during that time return the ASCII code of the key (we apply a mask)
    # otherwise return -1
    # 
    # the delay controls the speed of the video
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Probably clean the memory of "cap" object
cap.release()

# Straight forward
cv2.destroyAllWindows()

## **Counting number of frames**

In [ ]:
cap = cv2.VideoCapture(video_path)
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("Nb frames:", length)

512


## **Drawing landmarks**

Src: https://medium.com/@riddhisi238/real-time-pose-estimation-from-video-using-mediapipe-and-opencv-in-python-20f9f19c77a6<br>



In [ ]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Open the video file
cap = cv2.VideoCapture(video_path)

frame_number = 0
# csv_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with MediaPipe Pose
    result = pose.process(frame_rgb)

    # Draw the pose landmarks on the frame
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Add the landmark coordinates to the list and print them
        # write_landmarks_to_csv(result.pose_landmarks.landmark, frame_number, csv_data)

    # Display the frame
    cv2.imshow('MediaPipe Pose', frame)

    df = pd.DataFrame(csv_data)
    df.to_csv('landmarks.csv')

: 

In [4]:
cap = cv2.VideoCapture(video_path)

for i in range(10):
    if not(cap.isOpened()):
        break

    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Video", frame)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()